In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchinfo import summary
from torchvision import datasets, transforms

In [2]:
# 1. Configuration and Data Loading
BATCH_SIZE = 64
transform = transforms.Compose([
    transforms.ToTensor(),
    # Normalize with MNIST mean (0.1307) and std (0.3081)
    transforms.Normalize((0.1307,), (0.3081,))
])

In [3]:
# Download and load training data
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Download and load test data
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [4]:
# Use GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [5]:
# 2. Define the Neural Network Model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3)  #26 26
        self.pool1 = nn.MaxPool2d(kernel_size=2)  # 13 13

        self.conv2 = nn.Conv2d(8, 16, kernel_size=3)  #11 11
        # self.pool2 = nn.AdaptiveAvgPool2d(output_size=(3, 3))
        self.pool2 = nn.MaxPool2d(kernel_size=3)

        self.fc3 = nn.Linear(16 * 3 * 3, 10)

    def forward(self, x):
        # Flatten the input image from 28x28 to a 784-element vector
        x = F.relu(self.conv1(x))
        x = self.pool1(x)

        x = F.relu(self.conv2(x))
        x = self.pool2(x)

        x = x.view(-1, 16 * 3 * 3)
        x = self.fc3(x)
        # Apply log softmax for the final output (often used with NLLLoss, which is part of CrossEntropyLoss)
        return F.log_softmax(x, dim=1)


model = SimpleNN().to(device)

In [6]:
# 3. Define Loss and Optimizer
criterion = nn.NLLLoss()  # Negative Log-Likelihood Loss (suitable for log_softmax output)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
summary(model)

Layer (type:depth-idx)                   Param #
SimpleNN                                 --
├─Conv2d: 1-1                            80
├─MaxPool2d: 1-2                         --
├─Conv2d: 1-3                            1,168
├─MaxPool2d: 1-4                         --
├─Linear: 1-5                            1,450
Total params: 2,698
Trainable params: 2,698
Non-trainable params: 0

In [8]:
from datetime import datetime

writer = SummaryWriter(f'runs/init_{datetime.now().strftime("%Y_%m_%d-%H_%M_%S")}')

In [9]:
# 4. Training Loop
NUM_EPOCHS = 5

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()  # Clear gradients
        output = model(data)  # Forward pass
        loss = criterion(output, target)  # Calculate loss
        loss.backward()  # Backward pass (calculate gradients)
        optimizer.step()  # Update weights

        running_loss += loss.item()

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch + 1}/{NUM_EPOCHS} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
    avg_train_loss = running_loss / len(train_loader)
    writer.add_scalar('Loss/Train', avg_train_loss, epoch)

writer.flush()
writer.close()

Train Epoch: 1/5 [0/60000 (0%)]	Loss: 2.353880
Train Epoch: 1/5 [6400/60000 (11%)]	Loss: 0.605367
Train Epoch: 1/5 [12800/60000 (21%)]	Loss: 0.189975
Train Epoch: 1/5 [19200/60000 (32%)]	Loss: 0.306724
Train Epoch: 1/5 [25600/60000 (43%)]	Loss: 0.299493
Train Epoch: 1/5 [32000/60000 (53%)]	Loss: 0.156264
Train Epoch: 1/5 [38400/60000 (64%)]	Loss: 0.255736
Train Epoch: 1/5 [44800/60000 (75%)]	Loss: 0.227101
Train Epoch: 1/5 [51200/60000 (85%)]	Loss: 0.031764
Train Epoch: 1/5 [57600/60000 (96%)]	Loss: 0.095282
Train Epoch: 2/5 [0/60000 (0%)]	Loss: 0.346507
Train Epoch: 2/5 [6400/60000 (11%)]	Loss: 0.047977
Train Epoch: 2/5 [12800/60000 (21%)]	Loss: 0.100386
Train Epoch: 2/5 [19200/60000 (32%)]	Loss: 0.013483
Train Epoch: 2/5 [25600/60000 (43%)]	Loss: 0.097030
Train Epoch: 2/5 [32000/60000 (53%)]	Loss: 0.095211
Train Epoch: 2/5 [38400/60000 (64%)]	Loss: 0.090845
Train Epoch: 2/5 [44800/60000 (75%)]	Loss: 0.195904
Train Epoch: 2/5 [51200/60000 (85%)]	Loss: 0.130863
Train Epoch: 2/5 [57600/

In [10]:
model.eval()

SimpleNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (fc3): Linear(in_features=144, out_features=10, bias=True)
)

In [11]:
model.cpu()

SimpleNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (fc3): Linear(in_features=144, out_features=10, bias=True)
)

In [12]:
# tensorboard --logdir=runs

In [13]:
# 5. Testing the Model
test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)
accuracy = 100. * correct / len(test_loader.dataset)

print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
      f'({accuracy:.2f}%)\n')


Test set: Average loss: 0.0588, Accuracy: 9823/10000 (98.23%)



In [19]:
model.state_dict()

OrderedDict([('conv1.weight',
              tensor([[[[-0.4814, -0.6990, -0.7116],
                        [ 0.4219,  0.0650, -0.0044],
                        [ 0.5095,  0.2138,  0.7075]]],
              
              
                      [[[-0.2946,  0.3478,  0.4269],
                        [-0.2358, -0.0920,  0.4480],
                        [-0.0642, -0.5702,  0.1428]]],
              
              
                      [[[-0.5580, -0.1553,  0.1899],
                        [-0.3164, -0.4126, -0.4613],
                        [-0.4651, -0.1684, -0.6459]]],
              
              
                      [[[-0.1388, -0.6455,  0.0852],
                        [ 0.4127, -0.3864, -0.5679],
                        [ 0.5861,  0.2225, -0.3083]]],
              
              
                      [[[ 0.1478, -0.0027,  0.4175],
                        [-0.1326,  0.1175,  0.3983],
                        [-0.5018, -0.1197,  0.2888]]],
              
              
               

In [14]:
# Save the model (optional)
torch.save(model.state_dict(), "mnist_simple_nn.pth")

In [15]:
def export_onnx(model, onnx_path="dynamic_cnn.onnx"):
    dummy = torch.randn(1, 1, 28, 28)

    with torch.inference_mode():
        torch.onnx.export(
            model,
            dummy,
            onnx_path,
            opset_version=18,
            dynamo=False,
            export_params=True,
            do_constant_folding=True,
            input_names=["input"],
            output_names=["output"],
            dynamic_axes={
                "input": {0: "batch", 2: "height", 3: "width"},
                "output": {0: "batch"},
            },
        )

In [16]:
onnx_path = "model.onnx"
export_onnx(model, onnx_path)

C:\Users\nick\AppData\Local\Temp\ipykernel_7788\1920309358.py:5: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(


In [17]:
import onnx

onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)  # Check for model correctness

print("Model inputs:")
for input in onnx_model.graph.input:
    print(f"Input name: {input.name}")
    print(
        f"Input shape: {[(d.dim_value if d.HasField('dim_value') else d.dim_param) for d in input.type.tensor_type.shape.dim]}")

Model inputs:
Input name: input
Input shape: ['batch', 1, 'height', 'width']


In [18]:
# Take a look at model graph
# https://netron.app/